In [4]:
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
from sqlalchemy import create_engine, text


In [5]:
# =========================
# 0) DB 연결 설정
# =========================
DB_USER = "usedcar_user"
DB_PASSWORD = "usedcar_user"
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "usedcar_proj"
TABLE = "fact_car_listing"

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4"
)

In [6]:
df = pd.read_sql("SELECT * FROM fact_car_listing", engine)
df.shape

(2635, 11)

In [7]:
# 숫자형 컬럼 안전 변환
for c in ["year_int", "mileage_km", "price_manwon", "brand_id"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 분석에 필요한 값이 있는 행만
df_valid = df[
    df["price_manwon"].notna() &
    df["year_int"].notna() &
    df["mileage_km"].notna() &
    df["brand_id"].notna()
].copy()

df_valid.shape


(1750, 11)

In [9]:
def normalize_model_name(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.strip()
    s = re.sub(r"[^\w\s가-힣]", " ", s)   # 특수문자 제거
    s = s.upper()                        # 영문 대문자
    s = re.sub(r"\s+", " ", s).strip()  # 공백 정리
    return s

if "model_key" not in df_valid.columns:
    df_valid["model_key"] = (
        df_valid["model_name_raw"]
        .fillna("")
        .astype(str)
        .apply(normalize_model_name)
    )

df_valid[["model_name_raw","model_key"]].head()


,model_name_raw,model_key
0,현대 e-카운티 롱바디 25인승 자가용 슈퍼,현대 E 카운티 롱바디 25인승 자가용 슈퍼
1,현대 e-에어로타운 롱바디 34인승 자가용,현대 E 에어로타운 롱바디 34인승 자가용
2,현대 e-카운티 롱바디 25인승 자가용 슈퍼,현대 E 카운티 롱바디 25인승 자가용 슈퍼
3,현대 e-에어로타운 롱바디 34인승 자가용,현대 E 에어로타운 롱바디 34인승 자가용
4,현대 뉴스타렉스 캠핑카,현대 뉴스타렉스 캠핑카


In [15]:
def build_similarity_cluster(
    df: pd.DataFrame,
    target: dict,
    *,
    key_col: str = "model_key",
    year_col: str = "year_int",
    mileage_col: str = "mileage_km",
    price_col: str = "price_manwon",
    fuel_col: str = "fuel_type",
    # 룰베이스 가중치 (문헌 환산: 1년 ≈ 22,000km → beta ≈ 1/2.2)
    alpha_year: float = 1.0,
    beta_mile: float = 1.0 / 2.2,   # ≈0.4545, mileage_10k 단위
    fuel_same: float = 1.0,
    fuel_diff: float = 0.8,
    min_weight: float = 1e-6,
    top_k: int = 200
) -> pd.DataFrame:
    d = df.copy()

    # 0) 필수 컬럼 확인
    required = [price_col, year_col, mileage_col, key_col]
    for c in required:
        if c not in d.columns:
            raise ValueError(f"df에 '{c}' 컬럼이 없습니다.")

    # 1) 필수값(가격/연식/주행거리) 있는 행만
    d = d[d[price_col].notna() & d[year_col].notna() & d[mileage_col].notna()].copy()

    # 2) 하드 필터: 같은 모델 키만 (군집 경계)
    key_val = target.get(key_col, None)
    if key_val is None or str(key_val).strip() == "":
        raise ValueError(f"target['{key_col}'] 값이 필요합니다.")
    d = d[d[key_col] == key_val].copy()

    if len(d) == 0:
        return d.assign(weight=pd.Series(dtype=float))

    # 3) 거리 계산 (입력 차량 기준)
    try:
        t_year = float(target[year_col])
        t_mile = float(target[mileage_col])
    except Exception:
        raise ValueError(f"target에 '{year_col}', '{mileage_col}' 숫자값이 필요합니다.")

    d["year_diff"] = (d[year_col].astype(float) - t_year).abs()
    d["mile_diff_10k"] = (d[mileage_col].astype(float) - t_mile).abs() / 10000.0

    # 4) 기본 가중치: exp(-거리)
    d["base_weight"] = np.exp(-(alpha_year * d["year_diff"] + beta_mile * d["mile_diff_10k"]))

    # 5) 연료 가중치(있을 때만) - soft penalty
    if fuel_col in d.columns and fuel_col in target and pd.notna(target[fuel_col]):
        t_fuel = str(target[fuel_col])
        d["fuel_weight"] = np.where(d[fuel_col].astype(str) == t_fuel, fuel_same, fuel_diff)
    else:
        d["fuel_weight"] = 1.0

    # 6) 최종 가중치
    d["weight"] = d["base_weight"] * d["fuel_weight"]

    # 7) 너무 작은 가중치 제거
    if min_weight is not None:
        d = d[d["weight"] >= float(min_weight)].copy()

    # 8) 가장 유사한 top_k만
    d = d.sort_values("weight", ascending=False).head(int(top_k)).copy()

    return d


In [16]:
def weighted_quantile(values, quantiles, sample_weight=None):
    values = np.asarray(values, dtype=float)
    quantiles = np.asarray(quantiles, dtype=float)

    if sample_weight is None:
        sample_weight = np.ones_like(values, dtype=float)
    else:
        sample_weight = np.asarray(sample_weight, dtype=float)

    mask = np.isfinite(values) & np.isfinite(sample_weight) & (sample_weight > 0)
    values = values[mask]
    sample_weight = sample_weight[mask]

    if values.size == 0:
        return [None for _ in quantiles]

    sorter = np.argsort(values)
    values = values[sorter]
    sample_weight = sample_weight[sorter]

    cdf = np.cumsum(sample_weight)
    cdf = cdf / cdf[-1]

    return np.interp(quantiles, cdf, values).tolist()


def price_position_judgement(cluster_df: pd.DataFrame, target_price: float, price_col="price_manwon"):
    if cluster_df is None or len(cluster_df) == 0:
        return {"n": 0, "q1": None, "median": None, "q3": None, "label": "데이터 부족"}

    q1, med, q3 = weighted_quantile(
        cluster_df[price_col].values,
        [0.25, 0.50, 0.75],
        sample_weight=cluster_df["weight"].values
    )

    if q1 is None:
        return {"n": len(cluster_df), "q1": None, "median": None, "q3": None, "label": "데이터 부족"}

    p = float(target_price)
    if p < q1:
        label = "저렴"
    elif p > q3:
        label = "비쌈"
    else:
        label = "적정"

    return {"n": len(cluster_df), "q1": q1, "median": med, "q3": q3, "label": label}


In [17]:
target = {
    "model_key": df_valid["model_key"].value_counts().index[0],  # 가장 많은 모델로 테스트
    "year_int": int(df_valid["year_int"].median()),
    "mileage_km": float(df_valid["mileage_km"].median()),
    "price_manwon": float(df_valid["price_manwon"].median()),
    # 아래는 df에 컬럼이 있을 때만 사용
    # "fuel_type": "가솔린",
    # "transmission": "오토",
}

cluster = build_similarity_cluster(df_valid, target, top_k=200)
print("cluster size:", len(cluster))

judgement = price_position_judgement(cluster, target["price_manwon"])
judgement


cluster size: 14


{'n': 14,
 'q1': 2980.0,
 'median': 3888.1329878098068,
 'q3': 4384.066493904904,
 'label': '저렴'}

In [1]:
import pandas as pd
import numpy as np

# =========================
# 1️⃣ 데이터 로드
# =========================
df = pd.read_csv(
    r"C:\lecture\02_mysql_workspace\JangHanJae\_01_project_used_car\used_cars_bobaedream_final.csv"
)

print("shape:", df.shape)
display(df.head())

# =========================
# 2️⃣ 컬럼 확인
# =========================
print("\n[columns]")
print(df.columns.tolist())

# =========================
# 3️⃣ 결측치 비율
# =========================
na_ratio = df.isna().mean().sort_values(ascending=False)
display(na_ratio.head(20))

# =========================
# 4️⃣ 주요 컬럼 타입 확인
# =========================
cols = ["brand", "model_name", "price", "year", "mileage", "fuel"]
display(df[cols].info())

# =========================
# 5️⃣ 브랜드 분포
# =========================
brand_counts = df["brand"].value_counts()
display(brand_counts.head(20))

print("unique brands:", df["brand"].nunique())

# =========================
# 6️⃣ 모델 수 (raw)
# =========================
model_counts = df["model_name"].value_counts()
display(model_counts.head(20))

print("unique models:", df["model_name"].nunique())


shape: (2635, 13)


,brand,maker_no,model_name,price,year,mileage,fuel,region,seller_name,seller_type,reg_date,views,link
0,현대,49,현대 e-카운티 롱바디 25인승 자가용 슈퍼,"1,550만원",10/08,42만km,디젤,경기 화성시,김강섭,반복,02/03,181,https://www.bobaedream.co.kr/mycar/mycar_view....
1,현대,49,현대 e-에어로타운 롱바디 34인승 자가용,"4,350만원",15/11,11만km,디젤,경기 화성시,김강섭,반복,02/03,113,https://www.bobaedream.co.kr/mycar/mycar_view....
2,현대,49,현대 e-카운티 롱바디 25인승 자가용 슈퍼,"1,500만원",11/08,54만km,디젤,경기 화성시,김강섭,반복,02/03,80,https://www.bobaedream.co.kr/mycar/mycar_view....
3,현대,49,현대 e-에어로타운 롱바디 34인승 자가용,"1,950만원",12/06,54만km,디젤,경기 화성시,김강섭,반복,02/03,223,https://www.bobaedream.co.kr/mycar/mycar_view....
4,현대,49,현대 뉴스타렉스 캠핑카,"1,050만원",05/04,19만km,디젤,경기 화성시,김강섭,반복,02/03,320,https://www.bobaedream.co.kr/mycar/mycar_view....



[columns]
['brand', 'maker_no', 'model_name', 'price', 'year', 'mileage', 'fuel', 'region', 'seller_name', 'seller_type', 'reg_date', 'views', 'link']


seller_name    0.101708
fuel           0.000380
model_name     0.000000
maker_no       0.000000
brand          0.000000
year           0.000000
price          0.000000
mileage        0.000000
region         0.000000
seller_type    0.000000
reg_date       0.000000
views          0.000000
link           0.000000
dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2635 entries, 0 to 2634
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand       2635 non-null   object
 1   model_name  2635 non-null   object
 2   price       2635 non-null   object
 3   year        2635 non-null   object
 4   mileage     2635 non-null   object
 5   fuel        2634 non-null   object
dtypes: object(6)
memory usage: 123.6+ KB


None

brand
벤츠            554
현대            462
기아            430
포르쉐           280
BMW           194
제네시스          179
쉐보레/대우         80
랜드로버           78
르노코리아(삼성)      72
KG모빌리티(쌍용)     60
아우디            50
포드             49
지프             27
토요타            23
재규어            23
테슬라            15
렉서스            14
미니             13
폭스바겐           11
볼보             10
Name: count, dtype: int64

unique brands: 22


model_name
벤츠 G63 AMG                           34
벤츠 마이바흐 S580 4매틱                     30
벤츠 S580L 4매틱                         22
현대 그랜드스타렉스 캠핑카                       22
포르쉐 카이엔 3.0                          19
벤츠 마이바흐 GLS 600 4매틱 마누팍투어            18
벤츠 S450L 4매틱                         16
제네시스 더 올 뉴 G80 2.5 터보 AWD            15
벤츠 마이바흐 GLS 600 4매틱                  15
벤츠 S500L 4매틱                         14
랜드로버 레인지로버 4.4 P530 LWB 오토바이오그래피     14
포르쉐 카이엔 3.0 쿠페                       14
제네시스 더 올 뉴 G80 2.5 터보                13
기아 카니발 4세대 3.5 가솔린 9인승 하이리무진 시그니처    13
포르쉐 911 터보 S 카브리올레                   13
현대 e-카운티 캠핑카                         12
포르쉐 911 카레라 4S 카브리올레                 12
제네시스 GV80 3.5 터보 AWD 5인승             11
제네시스 GV80 2.5 터보 AWD 5인승             10
포르쉐 911 카레라 4 GTS 카브리올레              10
Name: count, dtype: int64

unique models: 1573


In [2]:
import re

def normalize_model_name(name: str):
    if not isinstance(name, str):
        return ""
    s = name.strip()
    s = re.sub(r"[^\w\s가-힣]", " ", s)
    s = s.upper()
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["model_key"] = df["model_name"].apply(normalize_model_name)

print("unique model_key:", df["model_key"].nunique())

model_key_counts = df["model_key"].value_counts()
display(model_key_counts.head(30))


unique model_key: 1466


model_key
벤츠 G63 AMG                              34
벤츠 마이바흐 S580 4매틱                        30
벤츠 S580L 4매틱                            22
현대 그랜드스타렉스 캠핑카                          22
벤츠 마이바흐 GLS 600 4매틱 마누팍투어               21
포르쉐 카이엔 3 0                             19
기아 카니발 4세대 3 5 가솔린 9인승 하이리무진 시그니처       17
벤츠 마이바흐 GLS 600 4매틱                     17
벤츠 S450L 4매틱                            16
제네시스 더 올 뉴 G80 2 5 터보 AWD               15
제네시스 더 올 뉴 G80 2 5 터보                   15
제네시스 GV80 2 5 터보 AWD 5인승                15
벤츠 S500L 4매틱                            14
랜드로버 레인지로버 4 4 P530 LWB 오토바이오그래피        14
포르쉐 카이엔 3 0 쿠페                          14
포르쉐 911 터보 S 카브리올레                      13
포르쉐 911 카레라 4S 카브리올레                    12
현대 E 카운티 캠핑카                            12
제네시스 GV80 3 5 터보 AWD 5인승                12
기아 더 뉴 카니발 4세대 1 6 터보 HEV 9인승 그래비티      11
포르쉐 911 카레라 4 GTS 카브리올레                 10
벤츠 마이바흐 S680 4매틱                         9
벤츠 S63L AMG 4매틱                          9
포

In [3]:
for n in [10, 30, 50, 100]:
    cnt = (model_key_counts >= n).sum()
    print(f"{n}대 이상 model_key 수:", cnt)


10대 이상 model_key 수: 21
30대 이상 model_key 수: 2
50대 이상 model_key 수: 0
100대 이상 model_key 수: 0


In [4]:
print("brand null ratio:", df["brand"].isna().mean())
print("brand unique:", df["brand"].nunique())

display(df["brand"].value_counts(dropna=False).head(20))


brand null ratio: 0.0
brand unique: 22


brand
벤츠            554
현대            462
기아            430
포르쉐           280
BMW           194
제네시스          179
쉐보레/대우         80
랜드로버           78
르노코리아(삼성)      72
KG모빌리티(쌍용)     60
아우디            50
포드             49
지프             27
토요타            23
재규어            23
테슬라            15
렉서스            14
미니             13
폭스바겐           11
볼보             10
Name: count, dtype: int64

In [5]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["mileage"] = pd.to_numeric(df["mileage"], errors="coerce")

display(df[["price","year","mileage"]].describe())


,price,year,mileage
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [6]:
import re

# -------------------------
# price: "1,550만원" → 1550
# -------------------------
def parse_price(x):
    if pd.isna(x):
        return None
    x = str(x)
    x = re.sub(r"[^\d]", "", x)  # 숫자만
    return int(x) if x else None


# -------------------------
# year: "10/08" → 2010
# -------------------------
def parse_year(x):
    if pd.isna(x):
        return None
    x = str(x)
    if "/" in x:
        y = x.split("/")[0]
        y = int(y)
        return 2000 + y if y < 30 else 1900 + y
    return None


# -------------------------
# mileage: "42만km" → 420000
# -------------------------
def parse_mileage(x):
    if pd.isna(x):
        return None
    x = str(x)

    if "만" in x:
        num = re.sub(r"[^\d]", "", x)
        return int(num) * 10000 if num else None

    num = re.sub(r"[^\d]", "", x)
    return int(num) if num else None


# =========================
# 변환 적용
# =========================
df["price_num"] = df["price"].apply(parse_price)
df["year_num"] = df["year"].apply(parse_year)
df["mileage_num"] = df["mileage"].apply(parse_mileage)

display(df[["price","price_num","year","year_num","mileage","mileage_num"]].head())


,price,price_num,year,year_num,mileage,mileage_num
0,NaN,None,NaN,None,NaN,None
1,NaN,None,NaN,None,NaN,None
2,NaN,None,NaN,None,NaN,None
3,NaN,None,NaN,None,NaN,None
4,NaN,None,NaN,None,NaN,None
